# VIS: Model \#2
## Siamese Fusion Discrimination Network for Video-Audio Matching

In [1]:
COLAB = True

In [2]:
if COLAB:
    from tqdm.notebook import tqdm
    from google.colab import drive

    drive.mount('/content/drive')

    %cd /content/drive/MyDrive/MIT6-8300_Computer_Vision/Visually-Indicated-Sounds/
else:
    from tqdm import tqdm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/MIT6-8300_Computer_Vision/Visually-Indicated-Sounds


In [3]:
from dataloader import VATransform

RANDOM_SEGMENT_SECONDS = 3
IMG_DOWN_SAMPLE_FACTOR = 5
FRAME_DOWN_SAMPLE_FACTOR = 6

N_EPOCHS = 8
TRAIN_DATASET_SUBSAMPLE_SIZE = 780
VAL_DATASET_SUBSAMPLE_SIZE = 96
TEST_DATASET_SUBSAMPLE_SIZE = 96
RESAMPLED_BATCH_NUMBER = 60
RESAMPLED_BATCH_SIZE = 30
BATCH_SIZE = 4
LEARNING_RATE = 0.0035

STARTER_MODEL = None

MODEL_NAME = 'fusion50_6'

transforms = [VATransform.RANDOM_SEGMENT, 
                    VATransform.IMG_DOWNSAMPLE,
                    VATransform.FRAME_DOWNSAMPLE,
                    VATransform.DATA_AUGMENT]

transform_args = {'random_segment_seconds': RANDOM_SEGMENT_SECONDS,
                        'img_downsample_factor': IMG_DOWN_SAMPLE_FACTOR,
                        'frame_downsample_factor': FRAME_DOWN_SAMPLE_FACTOR}

In [4]:
import os
import time
import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import copy as copy

from dataloader import VideoAudioDataset, ResamplingSampler

from constants import AUDIO_SAMPLE_RATE

# !! Put data file location in file `data_filepath`
# If file `data_filepath` does not exist, assume data is located in root
filepath = 'vis-data-256/vis-data-256/'

if os.path.isfile('data_filepath'):
    with open('data_filepath', 'r') as f:
        filepath = f.readline() + filepath

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
print("Active device: ", device)

Active device:  cuda


## Model structure

In [5]:
class FusionVIS(nn.Module):
    def __init__(self):
        super(FusionVIS, self).__init__()

        # audio preprocessing
        self.audio_preprocess = nn.Sequential(
            MelSpectrogram(sample_rate=AUDIO_SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128),
            AmplitudeToDB()
        )

        # resnet backbone
        self.backbone = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.backbone.fc = nn.Identity()

        # define convolutional layers
        # self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        # self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        # self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        # self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # video

        # define fully connected layers
        self.fc1 = nn.Linear(in_features=4096, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=1)

    def forward(self, video, audio):
      with torch.no_grad():
        batch_size, seq_len, c, h, w = video.size()
        video = video.view(batch_size*seq_len, c, h, w)

        # audio preprocessing
        spectrogram = self.audio_preprocess(audio)
        spec_3 = spectrogram.unsqueeze(1).repeat(1, 3, 1, 1) # Don't forget to unsqueeze(1) to keep batch_size!

        # backbone
        video = self.backbone(video)
        audio_feat = self.backbone(spec_3)
          
        # print("Audio postprocessing: (nothing to do)")
        # print(audio_feat.shape)

        # print("Video postprocessing:")
        # print(video.shape)
        video = video.reshape(batch_size, seq_len, -1) # This is debatable: we are forcing (batch_size ,X) from arbitrary shape - labels are mixed??
        # print(video.shape)
        video_feat = torch.max(video, dim=1)[0]
        # print(video_feat.shape)

        # concatenation
        # print("Final concat:")
        # print(video_feat.shape)
        # print(audio_feat.shape)
        fusion = torch.cat([video_feat, audio_feat], dim=1)

      fusion = self.fc1(fusion)
      fusion = F.relu(fusion)
      fusion = self.fc2(fusion)
      fusion = F.relu(fusion)
      fusion = self.fc3(fusion)
      fusion = F.sigmoid(fusion)

      return fusion.squeeze()

In [6]:
fusion_model = FusionVIS().to(device)

## Training

In [7]:
# Dataset setup

train_filenames = np.load('datasets/train_dataset.npy')
val_filenames = np.load('datasets/val_dataset.npy')
test_filenames = np.load('datasets/test_dataset.npy')

# File '2015-09-12-04-15-35-1_mic.wav' is corrupted
train_filenames = train_filenames[train_filenames[:, 1] != '2015-09-12-04-15-35-1_mic.wav']

train_filenames = train_filenames[:TRAIN_DATASET_SUBSAMPLE_SIZE,]
val_filenames = val_filenames[:VAL_DATASET_SUBSAMPLE_SIZE,]
test_filenames = test_filenames[:TEST_DATASET_SUBSAMPLE_SIZE,]

print("Training set size: ", train_filenames.shape)
print("Validation set size: ", val_filenames.shape)
print("Test set size: ", test_filenames.shape)

train_dataset = VideoAudioDataset(train_filenames, device, filepath_prefix=filepath, transform=transforms, **transform_args)
val_dataset = VideoAudioDataset(val_filenames, device, filepath_prefix=filepath, transform=transforms, **transform_args)
test_dataset = VideoAudioDataset(test_filenames, device, filepath_prefix=filepath, transform=transforms, **transform_args)

train_sampler = ResamplingSampler(train_dataset, batch_size=RESAMPLED_BATCH_SIZE, no_batches=RESAMPLED_BATCH_NUMBER, replacement=True)

train_loader = DataLoader(train_dataset, batch_sampler=train_sampler)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Training set size:  (780, 3)
Validation set size:  (96, 3)
Test set size:  (96, 3)


In [8]:
# Starting point
if STARTER_MODEL != None:
    fusion_model.load_state_dict(torch.load('model_weights/'+STARTER_MODEL+'_weights.pt'))           

In [9]:
dataloaders = {'train':train_loader, 'val':val_loader}

criterion = nn.BCELoss()
optimizer= optim.Adam(fusion_model.parameters(), lr=LEARNING_RATE)

since = time.time()

train_arr_loss = []
train_arr_acc = []
val_arr_loss = []
val_arr_acc = []

best_model_wts = copy.deepcopy(fusion_model.state_dict())
best_acc = 0.0

for epoch in range(N_EPOCHS):

  print('Epoch {}/{}'.format(epoch + 1, N_EPOCHS))
  print('-' * 10)

  for phase in ['train', 'val']:

    if phase == 'train':
      fusion_model.train()
    else:
      fusion_model.eval()

    c_loss = 0.0
    c_total = 0
    c_corrects = 0

    loader = dataloaders[phase]

    for video_feat, audio_feat, label in tqdm(dataloaders[phase]):
      video_feat = video_feat.to(device)
      audio_feat = audio_feat.to(device)
      label = label.to(device)

      optimizer.zero_grad()

      with torch.set_grad_enabled(phase == 'train'):

        output = fusion_model(video_feat, audio_feat)
        loss = criterion(output, label.float())

        pred = torch.round(output)

        if phase == 'train':
          loss.backward()
          optimizer.step()

      c_loss += loss.item()
      c_total += label.size(0)
      c_corrects += (pred == label).sum().item()

      torch.cuda.empty_cache()

    epoch_loss = c_loss / c_total
    epoch_acc = 1.0*c_corrects / c_total

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    if phase == 'val' and epoch_acc > best_acc:
      best_acc = epoch_acc
      best_model_wts = copy.deepcopy(fusion_model.state_dict())
    if phase == 'train':
      train_arr_loss.append(epoch_loss)
      train_arr_acc.append(epoch_acc)
    if phase == 'val':
      val_arr_loss.append(epoch_loss)
      val_arr_acc.append(epoch_acc)

  print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

Epoch 1/8
----------


  0%|          | 0/60 [00:00<?, ?it/s]

train Loss: 0.0120 Acc: 0.8578


  0%|          | 0/24 [00:00<?, ?it/s]

val Loss: 0.1805 Acc: 0.4688

Epoch 2/8
----------


  0%|          | 0/60 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# RENAME SAVE FILE AFTER SAVE!
torch.save(best_model_wts, 'model_weights/'+MODEL_NAME+'_weights.pt')
np.save('model_logs/loss_acc_logs/'+MODEL_NAME+'_train_arr_loss.npy', train_arr_loss)
np.save('model_logs/loss_acc_logs/'+MODEL_NAME+'_train_arr_acc.npy', train_arr_acc)
np.save('model_logs/loss_acc_logs/'+MODEL_NAME+'_val_arr_loss.npy', val_arr_loss)
np.save('model_logs/loss_acc_logs/'+MODEL_NAME+'_val_arr_acc.npy', val_arr_acc)

# write file MODEL_NAME_log.txt
with open('model_logs/'+MODEL_NAME+'_log.txt', 'w') as f:
    # write current date in format mm/dd/yyyy - hh:mm
    f.write(datetime.datetime.now().strftime("%m/%d/%Y - %H:%M")+"\n")
    f.write('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)+"\n")
    f.write('Best val Acc: {:4f}'.format(best_acc)+"\n")
    f.write('Model name: '+MODEL_NAME+"\n")
    f.write('Starter model: '+(STARTER_MODEL if STARTER_MODEL != None else 'None')+"\n")

In [ ]:
plt.plot(range(1,N_EPOCHS+1),train_arr_loss, label="Training Loss")
plt.plot(range(1,N_EPOCHS+1),val_arr_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.legend()
plt.grid()
plt.title("Fusion Model training")
plt.show()

plt.plot(range(1,N_EPOCHS+1),train_arr_acc, label="Training Accuracy")
plt.plot(range(1,N_EPOCHS+1),val_arr_acc, label="Validation Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.grid()
plt.title("Fusion Model training")
plt.show()

## Testing

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# fusion_model.load_state_dict(torch.load('model_weights/'+MODEL_NAME+'_weights.pt'))

In [ ]:
fusion_model.eval()

preds = []
correct = 0
total = 0

with torch.no_grad():
  for video_feat, audio_feat, label in tqdm(test_loader):
    video_feat = video_feat.to(device)
    audio_feat = audio_feat.to(device)
    label = label.to(device)
    output = fusion_model(video_feat, audio_feat)
    pred = torch.round(output)
    preds.append(pred.unsqueeze(0))
    total += label.size(0)
    correct += (pred == label).sum().item()
    torch.cuda.empty_cache()


In [ ]:
print(f"Accuracy on {total} test samples: {correct/total}")

In [ ]:
# log test accuracy
with open('model_logs/'+MODEL_NAME+'_log.txt', 'a') as f:
    f.write(f"Accuracy on {total} test samples: {correct/total}")

In [ ]:
preds

## Misc.